In [1]:
# Packages
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd

In [2]:
def get_news_urls(links_site):
    '''scrape the html of the site'''
    resp = requests.get(links_site)
 
    if not resp.ok:
        return None
 
    html = resp.content
 
    '''convert html to BeautifulSoup object'''
    soup = BeautifulSoup(html , 'lxml')
 
    '''get list of all links on webpage'''
    links = soup.find_all('a')
 
    urls = [link.get('href') for link in links]
    urls = [url for url in urls if url is not None]
 
    '''Filter the list of urls to just the news articles'''
    news_urls = [url for url in urls if '/article/' in url]
 
    return news_urls

In [3]:
def scrape_news_text(news_url):
 
    news_html = requests.get(news_url).content
 
    '''convert html to BeautifulSoup object'''
    news_soup = BeautifulSoup(news_html , 'lxml')
 
    paragraphs = [par.text for par in news_soup.find_all('p')]
    news_text = '\n'.join(paragraphs)
    print("Finished scrapping: ", news_url)
 
    return news_text

In [4]:
def scrape_all_articles(ticker , upper_page_limit = 5):
 
    landing_site = 'http://www.nasdaq.com/symbol/' + ticker + '/news-headlines'
 
    all_news_urls = get_news_urls(landing_site)
 
    current_urls_list = all_news_urls.copy()
 
    index = 2
 
    '''Loop through each sequential page, scraping the links from each'''
    while (current_urls_list is not None) and (current_urls_list != []) and \
        (index <= upper_page_limit):
 
        '''Construct URL for page in loop based off index'''
        current_site = landing_site + '?page=' + str(index)
        current_urls_list = get_news_urls(current_site)
 
        '''Append current webpage's list of urls to all_news_urls'''
        all_news_urls = all_news_urls + current_urls_list
 
        index = index + 1
        print("Done processing page: ", index)
 
    all_news_urls = list(set(all_news_urls))
 
    '''Now, we have a list of urls, we need to actually scrape the text'''
    all_articles = [scrape_news_text(news_url) for news_url in all_news_urls]
 
    return all_articles

In [5]:
def save_articles_to_files(articles, ticker):
    article_counter = 0
    for article in articles: 
        article_file = ticker + str(article_counter) + '.txt'
        file_name = os.path.join('article_data', ticker, article_file)

        f = open(file_name, 'w')
        f.write(article)
        f.close()
        article_counter += 1
        print("Processed Article Number: ", article_counter)


In [6]:
def get_processed_articles(articles):
    ad_string = "Enter up to 25 symbols separated by commas or spaces in the text box"
    intro = "Join the Nasdaq Community today and get free, instant access to portfolios, stock ratings, real-time alerts, and more!"
    processed_results = []
    for a in articles: 
        ad_string_pos = a.find(ad_string)
        intro_pos = a.find(intro)
        start_index = intro_pos + len(intro)
        end_index = ad_string_pos

        processed_results.append(a[start_index:end_index])
    
    return processed_results

In [7]:
def get_all_articles(tickers, num_pages_to_read):
    all_articles = []
    for ticker in tickers:
        raw_articles = scrape_all_articles(ticker, num_pages_to_read)
        processed_articles = get_processed_articles(raw_articles)
        all_articles += processed_articles 
        print("Finished processing articles for: ", ticker)
    return all_articles


In [8]:
def get_company_set(article, tickers):
    words_found = set([])
    ticker_set = set(tickers)
    article = article.split()
    for word in article:
        if word in ticker_set: 
            words_found.add(word)
    return words_found 
        
    

In [9]:
def update_graph(edge_graph, company_set):
    pairs = itertools.combinations(company_set, 2)

    for pair in pairs: 
        if pair in edge_graph or (pair[1], pair[0]) in edge_graph:
            processed_pair = pair if pair in edge_graph else (pair[1], pair[0])
            edge_graph[processed_pair] += 1

In [10]:
def read_articles_from_disk(data_directories):
    counter = 0
    articles = []
    for directory in data_directories: 
        file_list = os.listdir(directory)
        files_to_read = [os.path.join(directory, file) for file in file_list]
        for file in files_to_read: 
            with open(file, 'r') as content_file:
                content = content_file.read()
                articles.append(content)
    return articles
    

In [11]:
tickers = ['WMT', 'WBA', 'VZ', 'V', 'UTX', 'PG', 'DIS']
num_pages_to_read = 100

for t in tickers: 
    ticker = [t]
    all_articles = get_all_articles(ticker, num_pages_to_read)
    save_articles_to_files(all_articles, t)

Done processing page:  3
Done processing page:  4
Done processing page:  5
Done processing page:  6
Done processing page:  7
Done processing page:  8
Done processing page:  9
Done processing page:  10
Done processing page:  11
Done processing page:  12
Done processing page:  13
Done processing page:  14
Done processing page:  15
Done processing page:  16
Done processing page:  17
Done processing page:  18
Done processing page:  19
Done processing page:  20
Done processing page:  21
Done processing page:  22
Done processing page:  23
Done processing page:  24
Done processing page:  25
Done processing page:  26
Done processing page:  27
Done processing page:  28
Done processing page:  29
Done processing page:  30
Done processing page:  31
Done processing page:  32
Done processing page:  33
Done processing page:  34
Done processing page:  35
Done processing page:  36
Done processing page:  37
Done processing page:  38
Done processing page:  39
Done processing page:  40
Done processing pag

Finished scrapping:  https://www.nasdaq.com/article/why-tcs-stock-looks-really-compelling-on-this-dip-cm1102120
Finished scrapping:  https://www.nasdaq.com/article/5-reasons-jc-penney-is-becoming-a-penny-stock-cm1075734
Finished scrapping:  https://www.nasdaq.com/article/modis-jobs-deficit-jjs-largest-india-plant-idle-3-years-after-completion-20190519-00048
Finished scrapping:  https://www.nasdaq.com/article/consumer-sector-update-for-04042019-tsla-bpi-bzun-wmt-mcd-dis-cvs-ko-cm1124441
Finished scrapping:  https://www.nasdaq.com/article/consumer-sector-update-for-03262019-bbby-anfi-conn-wmt-mcd-dis-cvs-ko-cm1119882
Finished scrapping:  https://www.nasdaq.com/article/the-bull-markets-3-worst-dow-stocks-cm1112192
Finished scrapping:  https://www.nasdaq.com/article/3-dividend-stocks-perfect-for-retirees-cm1126294
Finished scrapping:  https://www.nasdaq.com/article/consumer-sector-update-for-01042019-tsla-calm-tile-wmt-mcd-dis-cvs-ko-cm1077463
Finished scrapping:  https://www.nasdaq.com/ar

Finished scrapping:  https://www.nasdaq.com/article/us-dollar-stores-warn-of-tariff-hit-after-reporting-upbeat-sales-20190530-00803
Finished scrapping:  https://www.nasdaq.com/article/3-key-takeaways-from-targets-fiscal-first-quarter-report-cm1154476
Finished scrapping:  https://www.nasdaq.com/article/chinas-tough-talk-leaves-trade-talks-with-us-in-limbo-20190517-00531
Finished scrapping:  https://www.nasdaq.com/article/consumer-sector-update-for-03272019-scvl-home-unf-wmt-mcd-dis-cvs-ko-cm1120630
Finished scrapping:  https://www.nasdaq.com/article/bernie-sanders-to-push-for-15hr-minimum-wage-at-walmart-shareholder-meeting-20190605-00002
Finished scrapping:  https://www.nasdaq.com/article/buy-walmart-wmt-stock-after-blowout-holiday-quarter-earnings-cm1103059
Finished scrapping:  https://www.nasdaq.com/article/consumer-sector-update-for-02192019-wmt-hmc-aap-wmt-mcd-dis-cvs-ko-cm1101662
Finished scrapping:  https://www.nasdaq.com/article/fedex-is-helping-big-box-retailers-compete-with-am

Finished scrapping:  https://www.nasdaq.com/article/consumer-sector-update-for-03072019-kr-igt-burl-wmt-mcd-dis-cvs-ko-cm1110747
Finished scrapping:  https://www.nasdaq.com/article/amazon-to-shut-pop-up-stores-focus-on-bookstores-amp-4-star-cm1110801
Finished scrapping:  https://www.nasdaq.com/article/7-trade-war-losers-set-to-become-big-2019-winners-cm1071854
Finished scrapping:  https://www.nasdaq.com/article/kirklands-stock-tumbles-on-soft-q4-sales-amp-lowered-outlook-cm1108762
Finished scrapping:  https://www.nasdaq.com/article/walmart-vs-amazon-who-will-win-in-2019-cm1134965
Finished scrapping:  https://www.nasdaq.com/article/consumer-sector-update-for-12142018-bel-dtea-cost-wmt-mcd-dis-cvs-ko-cm1069403
Finished scrapping:  https://www.nasdaq.com/article/earnings-reaction-history-wal-mart-stores-inc-500-follow-through-indicator-50-sensitive-cm1101517
Finished scrapping:  https://www.nasdaq.com/article/wall-st-rises-for-third-straight-day-on-data-earnings-20190516-01129
Finished sc

Finished scrapping:  https://www.nasdaq.com/article/10-dow-jones-stocks-holding-the-blue-chip-index-back-cm1126858
Finished scrapping:  https://www.nasdaq.com/article/why-walmart-stock-will-rally-to-115-in-2019-cm1102862
Finished scrapping:  https://www.nasdaq.com/article/costcos-decent-comparable-sales-run-continues-in-february-cm1112552
Finished scrapping:  https://www.nasdaq.com/article/pricesmart-inc-psmt-ex-dividend-date-scheduled-for-february-14-2019-cm1098701
Finished scrapping:  https://www.nasdaq.com/article/mothers-day-2019-10-high-tech-gifts-your-mom-will-love-cm1140116
Finished scrapping:  https://www.nasdaq.com/article/is-it-too-early-to-invest-in-food-delivery-companies-cm1160518
Finished scrapping:  https://www.nasdaq.com/article/theres-a-big-if-ahead-of-new-age-beverages-stock-climb-to-10-cm1139437
Finished scrapping:  https://www.nasdaq.com/article/consumer-sector-update-for-02042019-clx-pzza-gci-wmt-mcd-dis-cvs-ko-cm1092559
Finished scrapping:  https://www.nasdaq.com/

...

Finished scrapping:  https://www.nasdaq.com/article/the-average-us-millennial-watches-more-netflix-than-tv-cm1114882
Finished scrapping:  https://www.nasdaq.com/article/netflix-acquires-storybots-makes-push-into-childrens-programming-cm1146659
Finished scrapping:  https://www.nasdaq.com/article/is-ishares-msci-usa-equal-weighted-etf-eusa-a-strong-etf-right-now-cm1122926
Finished scrapping:  https://www.nasdaq.com/article/5-of-the-best-stocks-under-10-to-buy-right-now-cm1158923
Finished scrapping:  https://www.nasdaq.com/article/earnings-reaction-history-the-walt-disney-company-545-follow-through-indicator-23-sensitive-cm1093665
Finished scrapping:  https://www.nasdaq.com/article/why-disney-resorts-ridiculous-price-increase-makes-sense-cm1081197
Finished scrapping:  https://www.nasdaq.com/article/dow-movers-unh-ba-cm1117500
Finished scrapping:  https://www.nasdaq.com/article/are-tensions-thawing-between-amazon-and-google-cm1132560
Finished scrapping:  https://www.nasdaq.com/article/with

Finished scrapping:  https://www.nasdaq.com/article/lions-gate-offered-to-sell-starz-to-cbs-for-55-billion-20190517-00664
Finished scrapping:  https://www.nasdaq.com/article/netflix-stock-will-suffer-very-little-from-the-advent-of-disney-cm1139809
Finished scrapping:  https://www.nasdaq.com/article/-three-takeaways-from-the-q1-earnings-season-cm1142535
Finished scrapping:  https://www.nasdaq.com/article/3-reasons-2019-will-be-a-banner-year-for-disney-investors-cm1141853
Finished scrapping:  https://www.nasdaq.com/article/2-crucial-questions-netflix-should-answer-next-week-cm1128888
Finished scrapping:  https://www.nasdaq.com/article/disney-parks-lift-earnings-avengers-endgame-streaming-date-announced-20190508-01497
Finished scrapping:  https://www.nasdaq.com/article/the-zacks-analyst-blog-highlights-jpmorgan-disney-citigroup-pnc-financial-and-twitter-cm1122006
Finished scrapping:  https://www.nasdaq.com/article/stock-market-news-boeing-falls-on-crash-fears-disney-celebrates-captain-mar

Finished scrapping:  https://www.nasdaq.com/article/us-stockswall-street-falls-on-worries-over-trade-global-growth-20190409-00756
Finished scrapping:  https://www.nasdaq.com/article/3-things-that-can-go-wrong-for-disney-investors-on-wednesday-cm1143842
Finished scrapping:  https://www.nasdaq.com/article/wall-st-rebounds-as-uschina-trade-rhetoric-cools-20190514-00978
Finished scrapping:  https://www.nasdaq.com/article/why-disney-will-dominate-the-box-office-for-years-to-come-cm1148963
Finished scrapping:  https://www.nasdaq.com/article/why-sinclair-broadcast-stock-soared-monday-cm1143235
Finished scrapping:  https://www.nasdaq.com/article/mondays-vital-data-disney-netflix-and-wells-fargo-cm1129484
Finished scrapping:  https://www.nasdaq.com/article/us-stocks-on-the-movechevron-anadarko-jpmorgan-walt-disney-20190412-00404
Finished scrapping:  https://www.nasdaq.com/article/hulu-follows-netflix-with-its-own-price-change-cm1087686
Finished scrapping:  https://www.nasdaq.com/article/after-h

Processed Article Number:  228
Processed Article Number:  229
Processed Article Number:  230
Processed Article Number:  231
Processed Article Number:  232
Processed Article Number:  233
Processed Article Number:  234
Processed Article Number:  235
Processed Article Number:  236
Processed Article Number:  237
Processed Article Number:  238
Processed Article Number:  239
Processed Article Number:  240
Processed Article Number:  241
Processed Article Number:  242
Processed Article Number:  243
Processed Article Number:  244
Processed Article Number:  245
Processed Article Number:  246
Processed Article Number:  247
Processed Article Number:  248
Processed Article Number:  249
Processed Article Number:  250
Processed Article Number:  251
Processed Article Number:  252
Processed Article Number:  253
Processed Article Number:  254
Processed Article Number:  255
Processed Article Number:  256
Processed Article Number:  257
Processed Article Number:  258
Processed Article Number:  259
Processe

Processed Article Number:  496
Processed Article Number:  497
Processed Article Number:  498
Processed Article Number:  499
Processed Article Number:  500
Processed Article Number:  501
Processed Article Number:  502
Processed Article Number:  503
Processed Article Number:  504
Processed Article Number:  505
Processed Article Number:  506
Processed Article Number:  507
Processed Article Number:  508
Processed Article Number:  509
Processed Article Number:  510
Processed Article Number:  511
Processed Article Number:  512
Processed Article Number:  513
Processed Article Number:  514
Processed Article Number:  515
Processed Article Number:  516
Processed Article Number:  517
Processed Article Number:  518
Processed Article Number:  519
Processed Article Number:  520
Processed Article Number:  521
Processed Article Number:  522
Processed Article Number:  523
Processed Article Number:  524
Processed Article Number:  525
Processed Article Number:  526
Processed Article Number:  527
Processe